In [8]:
import pandas as pd
import numpy as np
import tkinter as tk
from tkinter.filedialog import askopenfilename
from scipy.optimize import nnls
import matplotlib.pyplot as plt

# global variables for cropping wavelength
lowerLimit=510
upperLimit=590

class Spectrum:
    data=np.array([])
    wavelength=np.array([])

def loadComponentesSpectra():
    '''load components spectrums for the analysis'''
    spectrumComponents = pd.read_csv (r'_components_spectra.csv')
    npComponents=spectrumComponents.to_numpy()
    npComponents[np.isnan(npComponents)]=0
    wavelengths=npComponents[:,0]
    oxyhemoglobin=npComponents[:,1]
    deoxyhemoglobin=npComponents[:,2]
    methemoglobin=npComponents[:,3]
    carboxyhemoglobin=npComponents[:,4]
    eumelanin=npComponents[:,5]
    yc1a=npComponents[:,6]
    yc2a=npComponents[:,7]

    components_spectra = {
            "wavelengths": wavelengths,
            "oxyhemoglobin": oxyhemoglobin,
            "deoxyhemoglobin": deoxyhemoglobin,
            "methemoglobi": methemoglobin,
            "carboxyhemoglobin": carboxyhemoglobin,
            "eumelanin": eumelanin,
            "yc1a": yc1a,
            "yc2a": yc2a
        }
    return components_spectra

COMPONENT=loadComponentesSpectra()


In [1]:
import pandas as pd
import numpy as np
import tkinter as tk
from tkinter.filedialog import askopenfilename
from scipy.optimize import nnls

# global variables for cropping wavelength
lowerLimit=510
upperLimit=590

class Spectrum:
    data=np.array([])
    wavelength=np.array([])

def loadComponentesSpectra():
    '''load components spectrums for the analysis'''
    spectrumComponents = pd.read_csv (r'_components_spectra.csv')
    npComponents=spectrumComponents.to_numpy()
    wavelengths=npComponents[:,0]
    oxyhemoglobin=npComponents[:,1]
    deoxyhemoglobin=npComponents[:,2]
    methemoglobin=npComponents[:,3]
    carboxyhemoglobin=npComponents[:,4]
    eumelanin=npComponents[:,5]
    yc1a=npComponents[:,6]
    yc2a=npComponents[:,7]

    components_spectra = {
            "wavelengths": wavelengths,
            "oxyhemoglobin": oxyhemoglobin,
            "deoxyhemoglobin": deoxyhemoglobin,
            "methemoglobi": methemoglobin,
            "carboxyhemoglobin": carboxyhemoglobin,
            "eumelanin": eumelanin,
            "yc1a": yc1a,
            "yc2a": yc2a
        }
    return components_spectra

def cropFunction(Spec):
    """crop the spectrum between lower limit and upper limit"""
    croppedSpectrum = Spectrum()
    croppedSpectrum.wavelength = Spec.wavelength[
        np.where(np.logical_and(lowerLimit <= Spec.wavelength, Spec.wavelength <= upperLimit))]
    croppedSpectrum.data = Spec.data[
        np.where(np.logical_and(lowerLimit <= Spec.wavelength, Spec.wavelength <= upperLimit))]
    return croppedSpectrum

def normalizeRef(Spec):
    """divide the spectrum by its standard deviation"""
    Spec.data=Spec.data/np.std(Spec.data)
    return Spec

def loadWhiteRef(referenceNameNothinInfront='int75_LEDON_nothingInFront.csv',
                 whiteReferenceName = 'int75_WHITEREFERENCE.csv',
                 skipRowsNothing=24, skipRowsWhite=24,wavelengthColumn=1,firstSpecColumn=4):
    ''' returns cropped (between 500 to 600) white reference and the wavelength'''
    refNothingInfront = pd.read_csv (referenceNameNothinInfront,sep=',',skiprows=skipRowsNothing).to_numpy()
    refWhite = pd.read_csv(whiteReferenceName,sep=',',skiprows=skipRowsWhite).to_numpy()
    refSpectrum = Spectrum()
    refSpectrum.wavelength = refWhite[:,wavelengthColumn]
    refSpectrum.data = np.mean(refWhite[:,firstSpecColumn:],axis=1)-np.mean(refNothingInfront[:,firstSpecColumn:],axis=1)
    croppedRef=cropFunction(refSpectrum)
    refCroppedNormalized=normalizeRef(croppedRef)
    return refCroppedNormalized

def loadDarkRef(skipRows=4,wavelengthColumn=0,firstSpecColumn=3):
    ''' returns cropped (between 500 to 600) dark reference and the wavelength'''
    filetypes = [("csv files", "*.csv")]
    csv_file_path = askopenfilename(title="select the dark reference .csv file",filetypes=filetypes)
    darkRef = pd.read_csv(csv_file_path, sep=',', skiprows=skipRows).to_numpy()
    darkRefSpec=Spectrum()
    darkRefSpec.data = np.mean(darkRef[:,firstSpecColumn:],axis=1)
    darkRefSpec.wavelength = darkRef[:,wavelengthColumn]
    croppedDarkRef=cropFunction(darkRefSpec)
    return croppedDarkRef

def loadSpectrum(skipRows=4,wavelengthColumn=0,firstSpecColumn=3):
    ''' returns dark reference and the wavelength'''
    filetypes = [("csv files", "*.csv")]
    csv_file_path = askopenfilename(title="select the spectrum .csv file", filetypes=filetypes)
    spectrumData = pd.read_csv(csv_file_path, sep=',', skiprows=skipRows).to_numpy()
    spec = Spectrum()
    spec.data = spectrumData[:, firstSpecColumn:]
    spec.wavelength = spectrumData[:, wavelengthColumn]
    croppedSpectrum = cropFunction(spec)
    return croppedSpectrum

def normalizeSpectrum(spec,darkRef):
    """returns the normalized spectrum for the data"""
    dRefTile = np.tile(darkRef.data, (spec.data.shape[1], 1))
    spectrumData=spec.data-dRefTile.T
    STDspectrum=np.std(spectrumData,axis=1)
    spectrumDataNormalized = Spectrum()
    spectrumDataNormalized.data = spectrumData/STDspectrum[:,None]
    spectrumDataNormalized.wavelength = spec.wavelength
    return spectrumDataNormalized

def find_nearest(array, value):
    """find the nearest value to a value in an array and returns the index"""
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx

def absorbanceSpectrum(refSpec,normalizedSpec):
    """calculate the absorbance spectrum using white reference and normalized spectrum"""
    modifiedData = np.zeros(normalizedSpec.data.shape)
    for i in range(normalizedSpec.wavelength.shape[0]):
        modifiedData[i,:] = refSpec.data[find_nearest(refSpec.wavelength, normalizedSpec.wavelength[i])]
    modifiedSpec=Spectrum()
    normalizedSpec.data[normalizedSpec.data==0]=0.0001
    modifiedSpec.data=np.log(np.divide(modifiedData, normalizedSpec.data, out=None, where=True, casting= 'same_kind',
                                order = 'K', dtype = None))
    modifiedSpec.wavelength = normalizedSpec.wavelength
    return modifiedSpec



def scattering(spec,bValue=1.5):
    """calculate the scattering spectrum"""
    return (spec.wavelength / 500) ** (-1 * bValue)

def reflection(spec):
    """calculate the reflection spectrum"""
    return np.squeeze(-np.log(spec.wavelength.reshape(-1, 1)))



def cropComponents(absorbanceSpectrum):
    """crop the components regarding the upper limit and lower limit wavelengths"""
    Components = loadComponentesSpectra()
    # absorbance, spectrumWavelength = absorbanceSpectrum()
    oxyhemoglobin = np.zeros(absorbanceSpectrum.wavelength.shape)
    deoxyhemoglobin = np.zeros(absorbanceSpectrum.wavelength.shape)
    melanin = np.zeros(absorbanceSpectrum.wavelength.shape)
    scat=scattering(absorbanceSpectrum)
    ref=reflection(absorbanceSpectrum)
    for i in range(absorbanceSpectrum.wavelength.shape[0]):
        oxyhemoglobin[i] = Components["oxyhemoglobin"][find_nearest(Components["wavelengths"],
                                                                    absorbanceSpectrum.wavelength[i])]
        deoxyhemoglobin[i] = Components["deoxyhemoglobin"][find_nearest(Components["wavelengths"],
                                                                      absorbanceSpectrum.wavelength[i])]
        melanin[i] = Components["eumelanin"][find_nearest(Components["wavelengths"],
                                                          absorbanceSpectrum.wavelength[i])]
    componentsCrop = {
        "scattering": scat,
        "reflection": ref,
        "oxyhemoglobin": oxyhemoglobin,
        "deoxyhemoglobin": deoxyhemoglobin,
        "melanin": melanin}
    return componentsCrop

def componentsToArray(components):
    """make an n*m array of the components to use as input of the nnls"""
    variables = np.ones(components["oxyhemoglobin"].shape)
    variables = np.vstack([variables, components["oxyhemoglobin"]])
    variables = np.vstack([variables, components["deoxyhemoglobin"]])
    variables = np.vstack([variables, components["melanin"]])
    variables = np.vstack([variables, components["scattering"]])
    variables = np.vstack([variables, components["reflection"]])
    print(variables.shape)
    print(variables)
    return variables

def getCoef(absorbance,variables):
    """apply nnls and get coefs"""
    allCoef=np.zeros([absorbance.data.shape[1],variables.shape[0]])
    for i in range(absorbance.data.shape[1]):
        coef=nnls(variables.T,absorbance.data[:,i],maxiter=200 )

        allCoef[i,:]=coef[0]
    print('all coef shape : ',allCoef.shape)
    print('all coefs :' , allCoef)
    return allCoef

def mainAnalysis ():
    """load data, do all the analysis, get coefs as concentration"""
    whiteRef=loadWhiteRef()
    darkRef=loadDarkRef()
    spectrums=loadSpectrum()
    normalizedSpectrum=normalizeSpectrum(spectrums,darkRef)
    absorbance=absorbanceSpectrum(whiteRef,normalizedSpectrum)
    croppedComponent=cropComponents(absorbance)
    features=componentsToArray(croppedComponent)
    print('features shape :', features.shape)
    coef=getCoef(absorbance,features)

    concentration = 100 * coef[:,1] /(coef[:,1]+coef[:,2])
    concentration[np.isnan(concentration)]=0

    return concentration

mainAnalysis()

#### This is for test

# def testAnalysis ():
#     """load data, do all the analysis, get coefs as concentration"""
#     whiteRef=loadWhiteRef()
#     darkRef=loadDarkRef()
#     spectrums=loadSpectrum()
#     normalizedSpectrum=normalizeSpectrum(spectrums,darkRef)
#     absorbance=absorbanceSpectrum(whiteRef,normalizedSpectrum)
#     croppedComponent=cropComponents(absorbance)
#
#     #test spectrum with oxyhemoglobin
#     testSpec=Spectrum()
#     testSpec.data=(10+2*croppedComponent['oxyhemoglobin']+3*croppedComponent['deoxyhemoglobin']+
#                    croppedComponent['melanin']+4*croppedComponent['scattering']+
#                    croppedComponent['reflection']).reshape(-1,1)
#     testSpec.wavelength=absorbance.wavelength
#
#     features=componentsToArray(croppedComponent)
#     coef=getCoef(testSpec,features)
#     concentration = 100 * coef[:,1] /(coef[:,1]+coef[:,2])
#     concentration[np.isnan(concentration)]=0
#     print(np.mean(concentration))
#     print(np.std(concentration))
#     print(concentration)
#     print(concentration.shape)
#     return concentration
#
# testAnalysis()

####### blood sample test

# def bloodTest ():
#     """load data, do all the analysis, get coefs as concentration"""
#     whiteRef=loadWhiteRef(referenceNameNothinInfront='/Users/elahe/Documents/Bloodsamples/int75_LEDON_nothingInFront.csv',
#                           whiteReferenceName='/Users/elahe/Documents/Bloodsamples/int75_WHITEREFERENCE.csv')
#     darkRef=loadDarkRef(skipRows=24,wavelengthColumn=1,firstSpecColumn=4)
#     darkRef.data[np.isnan(darkRef.data)] = 0
#     spectrums=loadSpectrum(skipRows=24,wavelengthColumn=1,firstSpecColumn=4)
#     spectrums.data[np.isnan(spectrums.data)] = 0
#     print(spectrums.data.shape)
#     normalizedSpectrum=normalizeSpectrum(spectrums,darkRef)
#     normalizedSpectrum.data[np.isnan(normalizedSpectrum.data)] = 0
#     absorbance=absorbanceSpectrum(whiteRef,normalizedSpectrum)
#     absorbance.data[np.isnan(absorbance.data)] = 0


#     croppedComponent=cropComponents(absorbance)
#     features=componentsToArray(croppedComponent)


#     coef=getCoef(absorbance,features)
#     concentration = 100 * coef[:,1] /(coef[:,1]+coef[:,2])
#     concentration[np.isnan(concentration)]=0
#     print(np.mean(concentration))
#     print(np.std(concentration))
#     print(concentration)
#     print(concentration.shape)
#     return concentration

# bloodTest()

(6, 80)
[[ 1.          1.          1.          1.          1.          1.
   1.          1.          1.          1.          1.          1.
   1.          1.          1.          1.          1.          1.
   1.          1.          1.          1.          1.          1.
   1.          1.          1.          1.          1.          1.
   1.          1.          1.          1.          1.          1.
   1.          1.          1.          1.          1.          1.
   1.          1.          1.          1.          1.          1.
   1.          1.          1.          1.          1.          1.
   1.          1.          1.          1.          1.          1.
   1.          1.          1.          1.          1.          1.
   1.          1.          1.          1.          1.          1.
   1.          1.          1.          1.          1.          1.
   1.          1.        ]
 [19.51       19.56       19.68       19.89       20.17       20.51
  20.94       21.5        22.99       2

array([14.71563174, 19.62806818, 18.00904996, 16.51203209, 20.08201812,
       21.96428549, 22.51229159, 17.81681424, 25.09996484, 18.66944174,
       14.08991208, 16.25050364, 12.35391541,  0.        ,  5.16497656,
       11.86124677,  7.4571426 ,  9.36802314,  7.95358199,  9.11001848,
       14.06507989,  7.82986932, 10.52052155, 13.89219919, 19.22819095,
       22.0759399 , 23.96330524, 25.301213  , 19.25601557, 27.09621971,
       27.62226309, 28.06474176, 22.30495367, 22.90451291, 24.27315764,
       20.00691868, 25.55555349, 23.51037245, 23.5421444 , 21.71687345,
       22.20858209, 25.35010985, 23.08784414, 22.23188131, 22.52836564,
       27.59213093, 24.62544356, 24.02117522, 28.65626555, 28.64237003,
       29.52868433, 31.09636333, 31.2297494 , 29.63124997, 24.23805544,
       23.11575893, 23.1068046 , 20.83567128,  7.1951506 , 10.26270826,
       16.12752364, 19.63431417, 25.23838667, 24.77837307, 22.73913091,
       23.7363625 , 25.61000244, 20.76967148, 23.25463345, 14.02